In [1]:
import typing

import numpy as np
import pandas as pd

from datasets import load_dataset

import tqdm 
import requests

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

import cltrier_nlp as nlp

In [2]:
SAMPLE_SIZE: int = 100

In [3]:
DATASET = load_dataset("stanfordnlp/imdb").shuffle()

In [4]:
def remote_encoding(batch: typing.List[str]) -> typing.List:
    embeds: typing.List[np.ndarray] = []

    for value in tqdm.tqdm(batch):
    
        try: 
            embed = np.array(requests.post(
                'https://inf.cl.uni-trier.de/embed/',
                json={'prompt': value}
            ).json()["response"])
            
        except Exception as _e:
            display(_e)
            embed = None
        
        embeds.append(embed)

    return embeds

In [5]:
ENCODERS: typing.Dict[str, typing.Dict[str, typing.Callable]] = {
    "tfidf": {
        "engine": (tfidf := TfidfVectorizer()),
        "embed_train": lambda x: tfidf.fit_transform(x),
        "embed_test": lambda x: tfidf.transform(x),
    },
    "tiny transformer (local)": {
        "engine": (transformer := nlp.encoder.Encoder()),
        "embed_train": lambda x: np.stack(
            [embed.detach().numpy() for embed in nlp.encoder.EncoderPooler()(transformer(x), form="sent_cls")]),
        "embed_test": lambda x: np.stack(
            [embed.detach().numpy() for embed in nlp.encoder.EncoderPooler()(transformer(x), form="sent_cls")]),
    },
    "sota transformer (remote)": {
        "embed_train": lambda x: remote_encoding(x),
        "embed_test": lambda x: remote_encoding(x),
    }
}

/Users/simon/Library/Caches/pypoetry/virtualenvs/cltrier-nlp-4sfjUWJz-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
CLASSIFIERS: typing.Dict[str, typing.Callable] = {
    "svc": LinearSVC,
    "random_forest": RandomForestClassifier,
    "ada_boost": AdaBoostClassifier,
    "decision_tree": DecisionTreeClassifier,
    "k_neighbors": KNeighborsClassifier,
    "mlp": MLPClassifier,
}

In [7]:
results: typing.List[pd.DataFrame] = []

for encoder_label, encoder in ENCODERS.items():

    embed_train = encoder["embed_train"](DATASET["train"][:SAMPLE_SIZE]["text"])
    embed_test = encoder["embed_test"](DATASET["test"][:SAMPLE_SIZE]["text"])

    results.append(
        pd.json_normalize(
            data=[
                classification_report(
                    DATASET["test"]["label"][:SAMPLE_SIZE],
                    (
                        classifier()
                        .fit(
                                embed_train, 
                                DATASET["train"]["label"][:SAMPLE_SIZE]
                            )
                        .predict(embed_test)
                    ),
                    zero_division=1.,
                    output_dict=True
                ) | {"classifier": classifier_label, "encoder": encoder_label}
                for classifier_label, classifier in CLASSIFIERS.items()
            ]
        )
        .set_index(["encoder", "classifier"], drop=True)
        .filter(
            items=[
                "accuracy",
                "macro avg.f1-score",
                "weighted avg.f1-score"
            ]
        )
        .sort_values(by="accuracy", ascending=False)
    )

/Users/simon/Library/Caches/pypoetry/virtualenvs/cltrier-nlp-4sfjUWJz-py3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/simon/Library/Caches/pypoetry/virtualenvs/cltrier-nlp-4sfjUWJz-py3.11/lib/python3.11/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/simon/Library/Caches/pypoetry/virtualenvs/cltrier-nlp-4sfjUWJz-py3.11/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/simon/Library/Caches/pypoetry/virtualenvs/cltrier-nlp-4sfjUWJz-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perc

In [8]:
pd.concat(results).sort_values(by="weighted avg.f1-score", ascending=False)

accuracy  macro avg.f1-score  \
encoder                   classifier                                    
sota transformer (remote) mlp                0.85            0.849624   
                          k_neighbors        0.84            0.840000   
                          random_forest      0.83            0.824941   
                          ada_boost          0.80            0.799277   
                          svc                0.80            0.798712   
tiny transformer (local)  mlp                0.73            0.728670   
sota transformer (remote) decision_tree      0.72            0.718988   
tfidf                     ada_boost          0.72            0.712644   
tiny transformer (local)  svc                0.69            0.689969   
tfidf                     svc                0.66            0.631076   
                          random_forest      0.66            0.626374   
tiny transformer (local)  random_forest      0.63            0.612930   
                          k_neighbors        0.61            0.609649   
tfidf                     mlp                0.61            0.588217   
                          k_neighbors        0.60            0.565972   
                          decision_tree      0.56            0.559824   
tiny transformer (local)  decision_tree      0.52            0.519231   
                          ada_boost          0.52            0.518266   

                                         weighted avg.f1-score  
encoder                   classifier                            
sota transformer (remote) mlp                         0.849624  
                          k_neighbors                 0.840000  
                          random_forest               0.824941  
                          ada_boost                   0.799277  
                          svc                         0.798712  
tiny transformer (local)  mlp                         0.728670  
sota transformer (remote) decision_tree               0.718988  
tfidf                     ada_boost                   0.712644  
tiny transformer (local)  svc                         0.689969  
tfidf                     svc                         0.631076  
                          random_forest               0.626374  
tiny transformer (local)  random_forest               0.612930  
                          k_neighbors                 0.609649  
tfidf                     mlp                         0.588217  
                          k_neighbors                 0.565972  
                          decision_tree               0.559824  
tiny transformer (local)  decision_tree               0.519231  
                          ada_boost                   0.518266